In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.covariance import LedoitWolf
import matplotlib.pyplot as plt

from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [132]:

# Step 1: Load data (TAIEX)
symbol = "0050.TW"
data = yf.download(symbol, start='2020-01-01')
returns = data[["Close"]].pct_change().dropna()
# returns = returns.resample("ME").sum()

[*********************100%***********************]  1 of 1 completed


In [133]:
returns["pct_power_5"] = np.power(returns.Close, 5)
tajm = 242
alpha = 2 / (tajm + 1)
returns['cumsum_bm'] = returns.Close.cumsum()
returns['ma_cumsum_bm'] = returns.cumsum_bm.rolling(60).mean()
returns['pct_for_calculating_profit'] = returns.Close.shift(-2)

returns["pct_power_5_ema"] = returns["pct_power_5"].ewm(alpha=alpha, adjust=False).mean()
returns['pct_power_5_ema_shift_1'] = returns.pct_power_5_ema.shift(1)

returns['signal'] = returns.apply(lambda row: 1 if row['pct_power_5_ema'] > row['pct_power_5_ema_shift_1']*1.01 or row['pct_power_5_ema'] > 0 else -1, axis=1)


returns['strat_pct'] = returns.signal * returns.pct_for_calculating_profit

k = 0
u = 1500

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1,
                     subplot_titles=symbol)

# fig.add_trace(go.Scatter(
#     x=returns.index[k:u],
#     y=returns.iloc[k:u]['strat_pct'].cumsum(),
#     mode='lines',
#     name='Cumulative Strategy %'
# ), row=1, col=1)
fig.add_trace(go.Scatter(
    x=returns.index[k:u],
    y=returns.iloc[k:u].cumsum_bm,
    mode='lines',
    name='Close Price'
), row=1, col=1)

# fig.add_trace(go.Scatter(
#     x=returns.index[k:u],
#     y=returns.iloc[k:u].ma_cumsum_bm,
#     mode='lines',
#     name='MA20'
# ), row=1, col=1)

fig.add_trace(go.Scatter(
    x=returns.index[k:u],
    y=returns.iloc[k:u]['signal'],
    mode='lines',
    name='Signal'
), row=2, col=1)

fig.update_layout(
    title=symbol,
    template="plotly_white",
    height=600
)

fig.update_xaxes(title_text="Index", row=2, col=1)
fig.update_yaxes(title_text="Signal", row=2, col=1)

fig.show()

In [19]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

future_period = 5  # Look-ahead period for price changes
returns['future_price'] = returns['Close'].shift(-future_period)
returns['price_change'] = (returns['future_price'] - returns['Close']) / returns['Close']

# Define target labels: 1 = Buy, -1 = Sell, 0 = Hold
threshold = 0.02  # 2% price change threshold
returns['target_label'] = returns['price_change'].apply(
    lambda x: 1 if x > threshold else (-1 if x < -threshold else 0)
)

# Drop rows with NaN values created due to shifting
returns = returns.dropna()

# Feature Engineering: Select features for the model
returns['signal_lag_1'] = returns['signal'].shift(1)
returns['signal_lag_2'] = returns['signal'].shift(2)
returns['pct_power_5_ema_diff'] = returns['pct_power_5_ema'] - returns['pct_power_5_ema_shift_1']
features = ['signal', 'signal_lag_1', 'signal_lag_2', 'pct_power_5_ema', 'pct_power_5_ema_diff']

# Prepare training and testing data
X = returns[features].fillna(0)
y = returns['target_label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

# Add predictions to the DataFrame
returns['predicted_label'] = model.predict(X)

# Calculate strat_pct using modeled signal
returns['strat_pct'] = returns['predicted_label'] * returns['pct_for_calculating_profit']

# Plot the results using Plotly
from plotly.subplots import make_subplots
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1,
                     subplot_titles=("Cumulative Strategy % and Close Price", "Signal Over Time"))

# Add cumulative strategy percentage and close price to the first subplot
fig.add_trace(go.Scatter(
    x=returns.index,
    y=returns['strat_pct'].cumsum(),
    mode='lines',
    name='Cumulative Strategy %'
), row=1, col=1)
fig.add_trace(go.Scatter(
    x=returns.index,
    y=returns['Close'].cumsum(),
    mode='lines',
    name='Close Price'
), row=1, col=1)

# Add signal values to the second subplot
fig.add_trace(go.Scatter(
    x=returns.index,
    y=returns['signal'],
    mode='lines',
    name='Signal'
), row=2, col=1)
fig.add_trace(go.Scatter(
    x=returns.index,
    y=returns['predicted_label'],
    mode='lines',
    name='Predicted Signal'
), row=2, col=1)

# Update layout
fig.update_layout(
    title="Combined Plot",
    xaxis_title="Index",
    yaxis_title="Value",
    template="plotly_white",
    height=600
)

fig.update_xaxes(title_text="Index", row=2, col=1)
fig.update_yaxes(title_text="Signal", row=2, col=1)

fig.show()


              precision    recall  f1-score   support

          -1       0.77      0.82      0.79       245
           0       0.00      0.00      0.00         2
           1       0.31      0.27      0.29        79

    accuracy                           0.68       326
   macro avg       0.36      0.36      0.36       326
weighted avg       0.66      0.68      0.67       326



/Users/tp_mini/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/tp_mini/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/tp_mini/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



 動能
 ARMA
 Sharpe(Time-Varying)
 VWAP

 低波动率指数：表现最为出色，不仅回报率最高，还显著降低了波动率和回撤幅度。
 高股息指数：在熊市中表现出色，具有防御性特征。
 动量指数：在牛市中表现更好，但在熊市中可能表现不佳。


In [ ]:
# 01-10 ret 是九號會出現的數據，是九號的收益，
# signal也會是九號出現的，因為是收盤價，所以十號的收益
# 所以建立一個column是ret.shift(-2)即可

In [466]:
# 牛市加碼
def rolling_1c(row):
    return 1
df_1 = pd.DataFrame(data['Close'])
df_1['ret'] = df_1.Close.pct_change()
df_1 = df_1.resample("ME").sum()
df_1['bm'] = df_1.ret.cumsum()
df_1['ret_for_cal'] =df_1.ret.shift(-2)
df_1['signal_val'] = df_1.ret
list_rolling_1c = [None] * len(df_1)
rolling = 5
for i in range(len(df_1)):
    if i < rolling: list_rolling_1c[i] = 0
    df_copy = df_1.iloc[i-rolling:i, :]
    d = np.corrcoef(df_copy.ret_for_cal, df_copy.signal_val)
    list_rolling_1c[i] = d[0, 1]
df_1['rolling_1c'] = list_rolling_1c

# df_1['rolling_1c_s1'] = df_1.rolling_1c.shift(1)
# df_1['signal'] = (df_1["rolling_1c"] < 0) & (df_1['rolling_1c_s1'] < 0)
# df_1['signal'] = df_1["signal"].apply(lambda x: 0 if x else 1)

# signal_conservative_m2_second_minus_1


df_1['signal'] = df_1["rolling_1c"].apply(lambda x: 2 if x < 0 else 0)
df_1['cumsum'] = (df_1.signal * df_1.ret_for_cal).cumsum()


def plotly_df(df, bar, line):
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=df.index, y=df[bar], name="val",
        yaxis="y2", marker=dict(color="red", opacity=0.6)))
    fig.add_trace(go.Scatter(
        x=df.index, y=df[line], mode="lines", name="ret",
        yaxis="y1", line=dict(color="blue")))
    fig.add_trace(go.Scatter(
        x=df.index, y=df['bm'], mode="lines", name="bm",
        yaxis="y1", line=dict(color="black")))
    
    fig.update_layout(title="1",xaxis=dict(title="Date"),
        yaxis=dict(title="Cumulative % Change",),
        yaxis2=dict(title="IR", overlaying="y", side="right"),
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    
    fig.show()
plotly_df(df_1, "rolling_1c", "cumsum")
# np.corrcoef(df_1.ret.fillna(0), df_1.signal.fillna(0))
